# 1. Introduction
Welcome to your third lab. In this lab, you will learn how to implement linear classifiers with some numerical data (Age, BMI, and Glucose) for predicting Diabetes_mellitus, which means whether the patient has diabetes(1) or not(0).

The dataset contains 25000 records for training set and 5000 for testing set.
Each instance has 3 features. The features contain Age, BMI, and Glucose.

There are three parts in this lab, including

  >Part 1: Implement the Perceptron
  >
  >Part 2: Implement Linear Discriminant Analysis (LDA)
  >
  >Part 3: Implement Linear Discriminant Analysis (LDA) classifier **using** Gaussian distributions and MAP estimation

Please think about the difference between the three classification methods in this lab. Write down your observations in the report.



# 2. Packages
All the packages that you need to finish this assignment are listed below.
*   numpy : the fundamental package for scientific computing with Python.
*   csv: a built-in Python module to handle CSV files for reading and writing tabular data.
*   pandas: a powerful data manipulation and analysis library for structured data, offering DataFrame objects for efficient handling of datasets
*   sklearn.metrics.f1_score: calculate the f1_score of the prediction

⚠️ **WARNING** ⚠️:
*   Please do not import any other packages in this lab.
*   np.random.seed(1) is used to keep all the random function calls consistent. It will help us grade your work. Please don't change the seed.

❗ **Important** ❗: Please do not change the code outside this code bracket.
```
### START CODE HERE ###
...
### END CODE HERE ###
```

## Import packages
> Note: You **cannot** import any other package in this lab

In [25]:
import numpy as np
import csv
import pandas as pd
from sklearn.metrics import f1_score
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
### START CODE HERE ###
# training_dataroot = '/drive/MyDrive/HW3/lab3_training.csv'
# testing_dataroot = '/drive/MyDrive/HW3/lab3_testing.csv'
training_dataroot = pd.read_csv('lab3_training.csv')
testing_dataroot = pd.read_csv('lab3_testing.csv')

# output_path_part1 = '/drive/MyDrive/HW3/lab3_part1.gsheet'
# output_path_part2 = '/drive/MyDrive/HW3/lab3_part2.gsheet'
# output_path_part3 = '/drive/MyDrive/ml learn/HW3/lab3_part3.gsheet'
output_path_part1 = 'lab3_part1.csv'
output_path_part2 = 'lab3_part2.csv'
output_path_part3 = 'lab3_part3.csv'
### END CODE HERE ###

# The example data that are used to test the calculation functions
X_exp = np.array([[-0.6415175074,	-0.2499581931,	-0.8246180885],
    [-1.00480699,	-0.4248545394,	0.1841706489],
    [-0.157131531,	-0.01062601622,	0.6666348277],
    [-1.125903484,	2.901126986,	0.3705772634],
    [1.174929904,	-0.5494975593,	0.2389961238],
    [-0.6415175074,	-0.1696408072,	0.6447046377],
    [-1.852482448,	-0.2256164258,	-0.6820718539],
    [-0.5809692604,	0.5687763118,	0.4363678332],
    [0.5088991865,	0.866833251,	-1.021989798],
    [1.05383341,	0.4590381523,	0.5679489729],
    [0.5694474336,	0.9588184702,	0.6885650176],
    [1.174929904,	0.09218273166,	-0.7588275187],
    [-0.2176797781,	-0.7169736941,	-0.6711067589],
    [0.9327369159,	-1.19479095,	-0.8026878985],
    [-1.368096472,	0.1146424265,	-0.4518048595],
    [-0.5809692604,	2.12232714,	0.3815423584],
    [-1.428644719,	2.01637356,	-0.627246379],
    [0.6905439277,	-0.5538426799,	-0.7917228036],
    [-1.125903484,	-1.205814174,	-0.616281284],
    [-0.2782280251,	-0.6836720793,	-0.8575133734]])
y_exp = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

## Split and preprocess data

In [68]:
# Read input csv to datalist
# train = pd.read_csv(training_dataroot)
# test = pd.read_csv(testing_dataroot)
train = training_dataroot
test = testing_dataroot

# split data
def SplitData(data):

  X_train = data.iloc[:20000, :3].values  # Get training features
  X_val = data.iloc[20000:, :3].values    # Get validation features
  y_train = data.iloc[:20000, 3].values   # Get training labels
  y_val = data.iloc[20000:, 3].values      # Get validation labels

  return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = SplitData(train)
X_test = test.iloc[:, :3].values

def StandardizeData(X_train, X_val, X_test):
  ### START CODE HERE ###
    # Calculate mean and standard deviation of the training set
    mean_train = X_train.mean(axis=0)
    std_train = X_train.std(axis=0)

    # Standardize the training set
    X_train_standardized = (X_train - mean_train) / std_train

    # Standardize validation set
    X_val_standardized = (X_val - mean_train) / std_train

    # Standardize test set
    X_test_standardized = (X_test - mean_train) / std_train

    return X_train_standardized, X_val_standardized, X_test_standardized
    ### END CODE HERE ###
X_train, X_val, X_test = StandardizeData(X_train, X_val, X_test)

print(type(X_train), type(X_val), type(X_test)) # <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
print(X_train.shape, X_val.shape, X_test.shape) # (20000, 3) (5000, 3) (5000, 3)
print(y_train.shape, y_val.shape) # (20000,) (5000,)

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(20000, 3) (5000, 3) (5000, 3)
(20000,) (5000,)


# Part 1 - Perceptron
In this part, you'll be implementing key components of a Perceptron. Your task is to complete the linear_combination and predict methods within the Perceptron class.

Here's what you need to focus on:

  >**The linear_combination function:**
  >
  >* This function calculates the weighted sum of input features.
  >* You'll need to use numpy's dot product function (np.dot) to multiply the input features with their corresponding weights.
  >* Remember to add the bias term (w_[0]) to your calculation.
  >
  >
  >**The predict function:**
  >
  >* This function determines the class prediction by calling linear_combination function.
  >* Use numpy's where function to implement the step function: return 1 if the weighted sum is greater than or equal to 0, and 0 otherwise.
  >
  >
  >**The fit function:**
  >
  >* Iterate through the training data.
  >* For each sample, use the predict method (which you've already implemented) to calculate ŷ (y_pred), the predicted value.
  >* Updating the weights by the weight update formula (Please note that our labels are 0 and 1, so the formula may be a little bit different with the one in the slides):
  >>$W_{i} = W_{i} + Δ(W_{i})$, where $Δ(W_{i}) = (y - \hat y) \times X_{i} $
  >* Remember to update both the feature weights (w_[1:]) and the bias term (w_[0]).
  >* Count the number of misclassifications in each iteration.

Hints:

* The weights (w_) are stored as a numpy array, with w_[0] as the bias term and w_[1:] as the weights for each feature.
* The input X is a 2D numpy array where each row represents a sample and each column a feature.

Reference: slides L5 p.8-16

**Please save the prediction result in a csv file lab3_part1.csv and upload to Kaggle**

In [69]:
class Perceptron(object):

  def __init__(self, X, n_iter=1):
    # initializing the weights to 0
    self.w_ = np.zeros(1 + X.shape[1])
    self.errors_ = []
    self.n_iter = n_iter

  def linear_combination(self, X):
    # calculate the sum of the weighted values
    ### START CODE HERE ###
    return np.dot(X, self.w_[1:]) + self.w_[0]  # Weighted sum with bias
    ### END CODE HERE ###


  def predict(self, X):
    # return the predicted value (ŷ) of X
    ### START CODE HERE ###
    return np.where(self.linear_combination(X) >= 0, 1, 0)  # Step function
    ### END CODE HERE ###

  def fit(self, X, y):
    print("Weights:", self.w_)

    # training the model n_iter times
    for _ in range(self.n_iter):
      error = 0

      # loop through each input
      for xi, yi in zip(X, y):
        ### START CODE HERE ###
        # Calculate ŷ (the predicted value)
        y_pred = self.predict(xi.reshape(1, -1))[0]  # Reshape xi for prediction

        # Update the weights (note that our labels are 0 and 1)
        # Wi = Wi + Δ(Wi) where Δ(Wi) = (y - ŷ) * Xi
        update = (yi - y_pred) * xi  # Calculate the update based on prediction error
        self.w_[1:] += update  # Update feature weights

        # Update the bias term (w_0)
        self.w_[0] += (yi - y_pred)  # Update bias

        # ŷ != y means mismatches
        error += int(y_pred != yi)  # Increment error if prediction is wrong
        ### END CODE HERE ###
      print(f"Errors in epoch {_}:", error)
      print("Updated Weights:", self.w_)

      self.errors_.append(error)

    return self


## Use the example data to test the weight caculation

Expected output:
> Weights: [0. 0. 0. 0.]
>
> Errors in epoch 0: 5
>
> Updated Weights: [-1.         -2.02260536  2.0821638  -0.75435559]

In [70]:
# X_exp has been standardized
perceptron = Perceptron(X_exp, n_iter=1)
perceptron.fit(X_exp, y_exp)

Weights: [0. 0. 0. 0.]
Errors in epoch 0: 5
Updated Weights: [-1.         -2.02260536  2.0821638  -0.75435559]


## Train and validate the model

In [71]:
### START CODE HERE ###
# you can change the iteration number if you want
perceptron = Perceptron(X_train, n_iter=10)
### END CODE HERE ###
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_val)
accuracy = np.mean(y_pred == y_val)
print("\nAccuracy:", accuracy)
print("\nF1 Score:", f1_score(y_val, y_pred))

Weights: [0. 0. 0. 0.]
Errors in epoch 0: 8009
Updated Weights: [-1.          0.73444263 -0.04382962  2.26842216]
Errors in epoch 1: 8015
Updated Weights: [-2.          1.08567614  1.02196519  2.3730904 ]
Errors in epoch 2: 8005
Updated Weights: [-1.          0.79831081 -0.42299194  1.8048286 ]
Errors in epoch 3: 7999
Updated Weights: [-2.          0.25538972  1.03835514  1.71244578]
Errors in epoch 4: 8046
Updated Weights: [0.         0.12773637 0.11695114 1.0385638 ]
Errors in epoch 5: 8055
Updated Weights: [-1.          0.73444263  0.06070078  1.54970935]
Errors in epoch 6: 8056
Updated Weights: [-1.          0.67057444 -0.06813246  2.08089048]
Errors in epoch 7: 8082
Updated Weights: [-1.        -0.5429211  0.0185557  4.2065669]
Errors in epoch 8: 8006
Updated Weights: [-1.          0.4151017  -0.589156    2.24843147]
Errors in epoch 9: 8043
Updated Weights: [-2.          0.38312609  0.85241037  1.51920247]

Accuracy: 0.5392

F1 Score: 0.3022410660205936


## Save the test result

In [72]:
y_pred = perceptron.predict(X_test)
with open(output_path_part1, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['id', 'diabetes_mellitus'])
  for i in range(len(y_pred)):
    writer.writerow([i, y_pred[i]])

# Part 2 - LDA

In this part, you'll be implementing key components of Linear Discriminant Analysis (LDA).

Here's what you need to focus on:

>**The fisher_discriminant function**
>* Compute the within-class scatter matrix (S_W)
>> $S_{W} = \sum_{n \in C_{1}}(X_{n}-m_{1})(X_{n}-m_{1})^{T} + \sum_{n \in C_{2}}(X_{n}-m_{2})(X_{n}-m_{2})^{T}$
>* Compute the between-class scatter matrix (S_B)
>> $S_{B} = (m_{2}-m_{1})(m_{2}-m_{1})^{T}$
>* Calculate the discriminant vector (w) using S_W and the class means
>> $w = S_{W}^{-1}(m_{2}-m_{1})$
>>
>> note that we define **$m_{1}$=class 0, $m_{2}$=class 1**
>* Normalize the discriminant vector
>
>>Hints:
>>* Remember to invert S_W using np.linalg.inv
>>* Normalize the final vector using np.linalg.norm
>
>**The boundary_calculation function**
>* This function calculates the decision boundary in the LDA-transformed space
>* Calculate the mean of each class in the transformed space
>* Compute the decision boundary as the average of these means
>
>>Hints:
>>* Use numpy's mean function (np.mean) with boolean indexing to separate classes
>
>**The lda_classifier function**
>* This function ties everything together to perform LDA classification.
>* Project the training and test data onto the LDA space
>* Calculate the decision boundary
>* Classify the test data based on this boundary
>
>>Hints:
>>* Use the dot product (.dot()) to project data onto the discriminant vector
>>* Implement the classification logic using a simple if-else statement

**Please save the prediction result in a csv file lab3_part2.csv and upload to Kaggle**


## Define Fisher's linear discriminant function

Reference:

slides L5 p.22-23

https://sthalles.github.io/fisher-linear-discriminant/

In [73]:
def fisher_discriminant(X, y):
    classes = np.unique(y)

    # Compute mean vectors for each class
    mean_vectors = [np.mean(X[y == cls], axis=0) for cls in classes]

    # Initialize within-class scatter matrix
    n_features = X.shape[1]  # Number of features
    S_W = np.zeros((n_features, n_features))  # Initialize within-class scatter matrix

    for cls, mean_vec in zip(classes, mean_vectors):
        class_scatter = np.cov(X[y == cls], rowvar=False)  # Covariance of each class
        n_cls = np.sum(y == cls)  # Number of samples in class
        S_W += class_scatter * n_cls  # Add class scatter, weighted by class size

    # Compute between-class scatter matrix
    overall_mean = np.mean(X, axis=0)  # Overall mean of the data
    S_B = np.zeros((n_features, n_features))  # Initialize between-class scatter matrix

    for cls, mean_vec in zip(classes, mean_vectors):
        n_cls = np.sum(y == cls)  # Number of samples in class
        mean_diff = (mean_vec - overall_mean).reshape(n_features, 1)  # Difference from overall mean
        S_B += n_cls * (mean_diff @ mean_diff.T)  # Add outer product of mean difference, weighted by class size

    # Compute the discriminant vector
    eig_vals, eig_vecs = np.linalg.eig(np.linalg.pinv(S_W).dot(S_B))  # Use pseudo-inverse to handle singular cases
    w = eig_vecs[:, np.argmax(eig_vals)]  # The direction of maximum variance (largest eigenvalue)

    # Normalize the discriminant vector
    w = w / np.linalg.norm(w)  # Normalize the vector

    return w


### Use the example data to test the weight caculation
Expected output:
> [ 0.37541286  0.64630924 -0.66434144]

In [74]:
# Get the discriminant
# X_exp has been standardized
W_exp = fisher_discriminant(X_exp, y_exp)
print(W_exp)

[ 0.40287754  0.6584693  -0.63569479]


## Implement a classifier

In [75]:
def boundary_calculation(X_train_lda, y_train):
  # Calculate the means and variances of the classes in the projected space
  ### START CODE HERE ###
  mean_class_0 = np.mean(X_train_lda[y_train == 0])
  mean_class_1 = np.mean(X_train_lda[y_train == 1])

  # Calculate the decision boundary (midpoint between the two means)
  decision_boundary = (mean_class_0 + mean_class_1) / 2
  ### END CODE HERE ###

  return decision_boundary

def lda_classifier(X_train, y_train, X_test):

  W = fisher_discriminant(X_train, y_train)

  ### START CODE HERE ###
  # Project onto the first discriminant
  X_train_lda = np.dot(X_train, W)  # 将训练集投影到 LDA 判别轴
  X_test_lda = np.dot(X_test, W)    # 将测试集投影到 LDA 判别轴

  # Calculate decision boundary
  decision_boundary = boundary_calculation(X_train_lda, y_train)

  # Predict class based on decision boundary
  y_pred = np.where(X_test_lda >= decision_boundary, 1, 0)  # 根据决策边界预测类别
  ### END CODE HERE ###

  return y_pred

### Use the example data to test the boundary calculation
Expected output:
> 0.5028438197095305

In [76]:
### START CODE HERE ###
W_exp = fisher_discriminant(X_exp, y_exp)  # 获取判别向量 W
X_exp_lda = np.dot(X_exp, W_exp)  # 将训练数据 X_exp 投影到 LDA 判别轴
### END CODE HERE ###
print(boundary_calculation(X_exp_lda, y_exp))

0.4988826354978516


## Train and validate the model

In [77]:
# Classify the projected test data
y_pred = lda_classifier(X_train, y_train, X_val)
# print(type(y_pred))
accuracy = np.mean(y_pred == y_val)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nF1 Score:", f1_score(y_val, y_pred))

Accuracy: 68.88%

F1 Score: 0.6847649918962723


## Save the test result

In [78]:
y_pred = lda_classifier(X_train, y_train, X_test)
with open(output_path_part2, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['id', 'diabetes_mellitus'])
  for i in range(len(y_pred)):
    writer.writerow([i, y_pred[i]])

# Part 3 - LDA with MAP

In this part, you're implementing a Linear Discriminant Analysis (LDA) classifier **using** Gaussian distributions and Maximum A Posterior (MAP) estimation.

>1. **Linear Discriminant Analysis (LDA):**
>LDA is a method that finds a linear combination of features that best separates two or more classes. It assumes that the classes are normally distributed with equal covariance matrices.
>
>2. **Gaussian Density Function:**
>The Gaussian (or normal) distribution is defined by its probability density function:
>
>>$f(x \mid \mu, \sigma^2) = \frac{1}{\sqrt{2\pi \sigma^2}} \exp \left( -\frac{(x - \mu)^2}{2\sigma^2} \right)$
>>
>>Where μ is the mean and σ² is the variance.
>
>3. **Maximum A Posteriori (MAP) Estimation:**
>MAP estimation seeks to find the most probable class given the observed data. It combines the likelihood of the data given the class (from the Gaussian density function) with the prior probability of the class.

Connecting LDA, Gaussian Distributions, and MAP:
>Step 1: LDA projects the data onto a lower-dimensional space that maximizes class separability (the **lda_classifier_map** function in the code)
>
>Step 2: After projection, we assume each class follows a Gaussian distribution in this new space. Computes the means, variances, and priors of each class in the LDA-projected space. (the **mean_variance_prior** function in the code)
>
>Step 3: Implement the Gaussian density function. (the **likelihood** function in the code)
>
>Step 4: Use MAP estimation (the **lda_classifier_map** function in the code)
>>* For each test point, calculate its likelihood of belonging to each class using the likelihood function (which you've already implemented).
>>* Multiply these likelihoods by the class priors to get quantities proportional to the posterior probabilities.
>>* Predict based on the highest posterior probability.

**Please save the prediction result in a csv file lab3_part3.csv and upload to Kaggle**

Reference: https://sthalles.github.io/fisher-linear-discriminant/


## Implement a classifier

Reference: [Linear Discriminant Analysis](https://chih-sheng-huang821.medium.com/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-lda%E5%88%86%E9%A1%9E%E6%BC%94%E7%AE%97%E6%B3%95-14622f29e4dc)

In [42]:
# Define a function to compute the likelihood
def mean_variance_prior(X_train_lda, y_train):

  ### START CODE HERE ###
  # Calculate the means and variances of the classes in the projected space
  mean_class_0 = np.mean(X_train_lda[y_train == 0])
  mean_class_1 = np.mean(X_train_lda[y_train == 1])

  variance_class_0 = np.var(X_train_lda[y_train == 0])
  variance_class_1 = np.var(X_train_lda[y_train == 1])

  # 计算先验概率
  prior_class_0 = np.mean(y_train == 0)
  prior_class_1 = np.mean(y_train == 1)
  ### END CODE HERE ###

  return mean_class_0, variance_class_0, prior_class_0, mean_class_1, variance_class_1, prior_class_1

def likelihood(mean, variance, x): # implement the Gaussian density distribution function
  ### START CODE HERE ###
  likelihood = (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-0.5 * ((x - mean) ** 2) / variance)
  ### END CODE HERE ###
  return likelihood

def lda_classifier_map(X_train, y_train, X_test):

  W = fisher_discriminant(X_train, y_train)

  ### START CODE HERE ###
  # Project onto the first discriminant
  X_train_lda = np.dot(X_train, W).reshape(-1)  # 将 X_train 投影到 LDA 判别轴
  X_test_lda = np.dot(X_test, W).reshape(-1)    # 将 X_test 投影到 LDA 判别轴
  ### END CODE HERE ###

  mean_class_0, variance_class_0, prior_class_0, mean_class_1, variance_class_1, prior_class_1 = mean_variance_prior(X_train_lda, y_train)

  ### START CODE HERE ###
  # Classify based on the maximum posterior probability
  predictions = []
  for x in X_test_lda:
      # 计算后验概率
      posterior_class_0 = likelihood(mean_class_0, variance_class_0, x) * prior_class_0
      posterior_class_1 = likelihood(mean_class_1, variance_class_1, x) * prior_class_1

      # 将样本分配给具有最大后验概率的类别
      if posterior_class_0 > posterior_class_1:
          predictions.append(0)
      else:
          predictions.append(1)

  ### END CODE HERE ###

  return np.array(predictions)

### Use the example data to test the likelihood calculation
If the differences between your output and expected output are only in last few decimal places, it's unlikely to affect the model's final results.

Expected output:
>means: 0.029797169482780564 0.9762686734207664
>
>variances: 0.36972519622481526 0.22878455291253338
>
>priors: 0.8421052631578947 0.15789473684210525
>
>likelihoods: 0.6560640840455648 0.11464725416998729

In [43]:
mean_class_0, variance_class_0, prior_class_0, mean_class_1, variance_class_1, prior_class_1 = mean_variance_prior(X_exp_lda[:19], y_exp[:19])
print("means:", mean_class_0, mean_class_1)
print("variances:", variance_class_0, variance_class_1)
print("priors:", prior_class_0, prior_class_1)
print("likelihoods:", likelihood(mean_class_0, variance_class_0, X_exp_lda[19]), likelihood(mean_class_1, variance_class_1, X_exp_lda[19]))

means: 0.019184256397725086 0.9807184481733625
variances: 0.3908276254241098 0.20163768973624152
priors: 0.8421052631578947 0.15789473684210525
likelihoods: 0.637065154158319 0.07521212637861983


## Train and validate the model

In [44]:
# Classify the projected test data
y_pred = lda_classifier_map(X_train, y_train, X_val)
# print(type(y_pred))
accuracy = np.mean(y_pred == y_val)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nF1 Score:", f1_score(y_val, y_pred))

Accuracy: 68.82%

F1 Score: 0.6875125275606334


## Save the test result

In [45]:
# Make predictions on the test data
y_pred = lda_classifier_map(X_train, y_train, X_test)
# Write the prediction to output csv
with open(output_path_part3, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['id', 'diabetes_mellitus'])
  for i in range(len(y_pred)):
    writer.writerow([i, y_pred[i]])